In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/ML/HAI817_Projet_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/ML/HAI817_Projet_test.csv')

In [ ]:
# Importation des différentes librairies, classes et fonctions utilespour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns

# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# librairies NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize 

 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
my_local_drive='/content/drive/MyDrive/ML/Prof/ML_FDS'

# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
#%cd $my_local_drive

#%pwd

# fonctions utilities (affichage, confusion, etc.)
from MyNLPUtilities import *

Concaténation des 2 df

In [ ]:
df_all = pd.concat([df_train,df_test])
df_all.head(10)


,public_id,text,title,our rating,ID
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,NaN
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture,NaN
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture,NaN
3,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false,NaN
4,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false,NaN
5,c03ed5db,Nation UPDATED 8:23 PM - K A B O O M! Governo...,K A B O O M! Governor and Secretary of State i...,other,NaN
6,61bd9a69,"November 23, 2019 The U.S. Food and Drug Admi...",FDA Shocking Study: Cells Used In Vaccines Con...,false,NaN
7,bb1999cc,"Trump confirms this was a bombing, not an acci...","Israel Hits Beirut with Nuclear Missile, Trump...",false,NaN
8,c1dc1ac6,In a show of anti-American sentiment that will...,Obama’s Daughters Caught on Camera Burning US ...,false,NaN
9,f2182a54,"FEMA camps, portable human cages, it's all rea...",Fields of human cages discovered in Caruthers ...,false,NaN


Vérification d'existence des valeurs nulls 

In [ ]:
# compter les valeurs manquantes dans chaque colonne
num_missing_values = df_all.isna().sum()
print(num_missing_values)

public_id      612
text             0
title           23
our rating       0
ID            1264
dtype: int64


Supression des classes : mixture et other 


In [ ]:
df_all = df_all[~df_all['our rating'].isin(['mixture', 'other'])]

Encodage des classes true et false

In [ ]:


df_all['classe'] = df_all['our rating'].map({'true': 1, 'false': 2})

vérification de désiquilibrage des classes 

In [ ]:
print(df_all["classe"].value_counts())

2    893
1    421
Name: classe, dtype: int64


Equilibrage des classes

In [ ]:
from sklearn.utils import resample
import pandas as pd



# Séparer les classes majoritaires et minoritaires
df_majority = df_all[df_all['classe'] == 2]
df_minority = df_all[df_all['classe'] == 1]

# Sous-échantillonner la classe majoritaire
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # Échantillonnage sans remplacement
                                   n_samples=len(df_minority), # Nombre d'échantillons égal à la classe minoritaire
                                   random_state=42)  # Pour la reproductibilité

# Combiner les classes majoritaire et minoritaire
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Afficher la taille du jeu de données équilibré
print("Taille du jeu de données équilibré : ", df_balanced.shape)

df_all = df_balanced
print(df_all['classe'].value_counts())

Taille du jeu de données équilibré :  (842, 6)
2    421
1    421
Name: classe, dtype: int64


Définition de la fonction  MyCleanText

In [ ]:
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def MyCleanText(X,
 lowercase=False, # mettre en minuscule
 removestopwords=False, # supprimer les stopwords
 removedigit=False, # supprimer les nombres
 getstemmer=False, # conserver la racine des termes
 getlemmatisation=False # lematisation des termes
 ):
    
    sentence = str(X)
    
    # suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)
    
    # decoupage en mots
    tokens = word_tokenize(sentence)
    
    if lowercase:
        tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]
    
    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]

    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]
    
    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]
    
    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
    
    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]

    sentence= ' '.join(words)

    return sentence


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Définition de la fonction TextNormalizer

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self,
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres
                 getstemmer=False,# racinisation des termes
                 getlemmatisation=False # lemmatisation des termes
                ):
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit
    
    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text, lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)
    
    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self


## **Evaluation de différents classifieurs**  


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# creation du tableau des différents classifieur 


models = []
models.append(('MultinomialNB',MultinomialNB()))
models.append(('LR', LogisticRegression(solver='lbfgs')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVM', SVC()))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import time


removestopwords=False, # suppression des stopwords

lowercase=False,# passage en minuscule

removedigit=False, # supprimer les nombres

getstemmer=False,# racinisation des termes

getlemmatisation=False # lemmatisation des termes

**Choix du meilleur classifieur avec les meilleurs parametres de prétraitement** 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import classification_report
import numpy as np
class Result:
    def __init__(self, name, score_accuracy, score_precision, score_rappel, score_mesure, temps, conf_mat):
        self.name = name
        self.score_accuracy = score_accuracy
        self.score_precision = score_precision
        self.score_rappel = score_rappel
        self.score_mesure = score_mesure
        self.temps = temps
        self.conf_mat = conf_mat 
def ma_fonction(removestopwords1, lowercase1, removedigit1, getstemmer1, getlemmatisation1):

    score_accuracy = 'accuracy'
    score_precision = 'precision'
    score_rappel = 'recall'
    score_mesure = 'f1'
    seed = 7        
    allresults = []
    results = []
    names = []

    X = df_all['title']
    y = df_all['classe']

    # Nous appliquons les pré-traitements sur X

    text_normalizer = TextNormalizer(removestopwords=removestopwords1, lowercase=lowercase1, removedigit=removedigit1, getstemmer=getstemmer1, getlemmatisation=getlemmatisation1)  
    # appliquer fit.transform pour réaliser les pré-traitements sur X
    X_cleaned = text_normalizer.fit_transform(X)

    # pour l'enchainer avec un tf-idf et obtenir une matrice
    tfidf = TfidfVectorizer()
    features = tfidf.fit_transform(X_cleaned).toarray()

    # attention ici il faut passer features dans cross_val_score plutôt que X

    for name, model in models:
        # cross validation en 10 fois
        kfold = KFold(n_splits=10, random_state=seed, shuffle=True)

        print("Evaluation de", name)
        start_time = time.time()
        # application de la classification
        cv_results = cross_val_score(model, features, y, cv=kfold, scoring=score_accuracy)
        cv_results_precision = cross_val_score(model, features, y, cv=kfold, scoring=score_precision)
        cv_results_rappel = cross_val_score(model, features, y, cv=kfold, scoring=score_rappel)
        cv_results_mesure = cross_val_score(model, features, y, cv=kfold, scoring=score_mesure)

        # Calculer la matrice de confusion
        y_pred = cross_val_predict(model, features, y, cv=kfold)
        conf_mat = confusion_matrix(y, y_pred)

        # Affichage du rapport de classification
        class_names = np.unique(y)
        print("Classification Report:")
        print(classification_report(y, y_pred, labels=class_names))

        # Affichage de la matrice de confusion
        print("Confusion Matrix:")
        print(conf_mat)

        thetime = time.time() - start_time






        result = Result(name, cv_results.mean(), cv_results_precision.mean(), cv_results_rappel.mean(), cv_results_mesure.mean(),  thetime, conf_mat)
        allresults.append(result)
        # pour affichage
        results.append(cv_results)
        names.append(name)
        print("%s : exactitude=%.3f, précision=%.3f, rappel=%.3f, mesure F1=%.3f in temps d'exécution=%.3f s" % (name, cv_results.mean(), cv_results_precision.mean(), cv_results_rappel.mean(), cv_results_mesure.mean(), thetime))


In [ ]:
#removestopwords, lowercase, removedigit, getstemmer,getlemmatisation
ma_fonction(True, True, True, True, True)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.61      0.63       421
           2       0.63      0.65      0.64       421

    accuracy                           0.63       842
   macro avg       0.63      0.63      0.63       842
weighted avg       0.63      0.63      0.63       842

Confusion Matrix:
[[258 163]
 [146 275]]
MultinomialNB : exactitude=0.633, précision=0.640, rappel=0.617, mesure F1=0.624 in temps d'exécution=1.193 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.65      0.64       421
           2       0.64      0.61      0.62       421

    accuracy                           0.63       842
   macro avg       0.63      0.63      0.63       842
weighted avg       0.63      0.63      0.63       842

Confusion Matrix:
[[275 146]
 [164 257]]
LR : exactitude=0.632, précision=0.629, rappel=0.656, mesure F1

In [ ]:
ma_fonction(True, True, True, False, False)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.64      0.64       421
           2       0.64      0.63      0.63       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[269 152]
 [155 266]]
MultinomialNB : exactitude=0.635, précision=0.636, rappel=0.646, mesure F1=0.638 in temps d'exécution=2.304 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.66      0.64       421
           2       0.64      0.62      0.63       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[276 145]
 [161 260]]
LR : exactitude=0.637, précision=0.633, rappel=0.664, mesure F1

In [ ]:
ma_fonction(True, True, True, True, False)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.62      0.63       421
           2       0.63      0.65      0.64       421

    accuracy                           0.63       842
   macro avg       0.63      0.63      0.63       842
weighted avg       0.63      0.63      0.63       842

Confusion Matrix:
[[259 162]
 [148 273]]
MultinomialNB : exactitude=0.632, précision=0.637, rappel=0.619, mesure F1=0.625 in temps d'exécution=1.048 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.62      0.65      0.64       421
           2       0.64      0.61      0.62       421

    accuracy                           0.63       842
   macro avg       0.63      0.63      0.63       842
weighted avg       0.63      0.63      0.63       842

Confusion Matrix:
[[275 146]
 [165 256]]
LR : exactitude=0.631, précision=0.627, rappel=0.656, mesure F1

In [ ]:
ma_fonction(True, True, False, True, False)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.62      0.63       421
           2       0.63      0.65      0.64       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[263 158]
 [148 273]]
MultinomialNB : exactitude=0.637, précision=0.640, rappel=0.626, mesure F1=0.630 in temps d'exécution=1.138 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.65      0.64       421
           2       0.64      0.62      0.63       421

    accuracy                           0.63       842
   macro avg       0.63      0.63      0.63       842
weighted avg       0.63      0.63      0.63       842

Confusion Matrix:
[[275 146]
 [162 259]]
LR : exactitude=0.634, précision=0.631, rappel=0.656, mesure F1

In [ ]:
ma_fonction(True, True, False, True, True)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.62      0.63       421
           2       0.63      0.65      0.64       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[263 158]
 [148 273]]
MultinomialNB : exactitude=0.637, précision=0.640, rappel=0.626, mesure F1=0.629 in temps d'exécution=1.114 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.66      0.65       421
           2       0.65      0.62      0.63       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[279 142]
 [162 259]]
LR : exactitude=0.639, précision=0.633, rappel=0.664, mesure F1

In [ ]:
ma_fonction(True, True, False, False, True)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.65      0.65      0.65       421
           2       0.65      0.64      0.65       421

    accuracy                           0.65       842
   macro avg       0.65      0.65      0.65       842
weighted avg       0.65      0.65      0.65       842

Confusion Matrix:
[[273 148]
 [150 271]]
MultinomialNB : exactitude=0.646, précision=0.647, rappel=0.653, mesure F1=0.646 in temps d'exécution=1.810 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.65      0.68      0.66       421
           2       0.66      0.62      0.64       421

    accuracy                           0.65       842
   macro avg       0.65      0.65      0.65       842
weighted avg       0.65      0.65      0.65       842

Confusion Matrix:
[[288 133]
 [158 263]]
LR : exactitude=0.654, précision=0.648, rappel=0.689, mesure F1

In [ ]:
ma_fonction(True, True, True, False, True)

Evaluation de MultinomialNB
Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.64      0.64       421
           2       0.64      0.64      0.64       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[268 153]
 [152 269]]
MultinomialNB : exactitude=0.638, précision=0.642, rappel=0.644, mesure F1=0.638 in temps d'exécution=1.271 s
Evaluation de LR
Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.67      0.65       421
           2       0.65      0.62      0.63       421

    accuracy                           0.64       842
   macro avg       0.64      0.64      0.64       842
weighted avg       0.64      0.64      0.64       842

Confusion Matrix:
[[282 139]
 [162 259]]
LR : exactitude=0.643, précision=0.639, rappel=0.678, mesure F1